In [1]:
links = [('Enercon E-family, 330 Kw to 7.5 MW, Wind Turbine Specification ',
  'http://www.enercon.de/p/downloads/EN_Productoverview_0710.pdf'),
 ('Enercon E-family, 330 Kw to 7.5 MW, Wind Turbine Specification ',
  'https://web.archive.org/web/20110516022444/http://www.enercon.de/p/downloads/EN_Productoverview_0710.pdf'),
 ('Wind Energy Handbook', 'https://books.google.com/books?id=4UYm893y-34C'),
 ('11 years of wind data shows surprising production decrease ',
  'http://ing.dk/artikel/11-aars-vinddata-afsloerede-overraskende-produktionsnedgang-163917')]

In [2]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from io import StringIO, BytesIO
from PyPDF2 import PdfReader, PdfFileReader
import urllib3
import html2text
import os
import time
from os import listdir
from os.path import isfile, join

http_urllib = urllib3.PoolManager()
tmp_path = './tmp'
downl_path = r'C:\Users\Anton\Downloads'
if not os.path.exists(tmp_path):
    os.makedirs(tmp_path)


for link_id, link in enumerate(links):
    title, url = link
    with open(f'{tmp_path}/{link_id}___{title}.txt', 'w', encoding='utf-8') as f:
        if 'pdf' == url[-3:]:
            response = http_urllib.request('GET', url)
            try:
                if str(response.status).strip() == '200':
                    memoryFile = BytesIO(response.data)
                    pdfFile = PdfFileReader(memoryFile)
                    answer = ''
                    for pageNum in range(pdfFile.getNumPages()):
                        currentPage = pdfFile.getPage(pageNum)
                        answer += currentPage.extract_text() + '\n'
                    f.write(answer)
                else:
                    pass
            except:
                driver = webdriver.Chrome(r'C:\Users\Anton\Desktop\chromedriver.exe')
                driver.get('https://pdfmyurl.com/')
                a = driver.find_element(By.ID, 'url')
                a.send_keys(url)
                button_element = driver.find_element(By.NAME, "submit")
                button_element.click()
                time.sleep(30)
                driver.close()
                
                pdf_saved_path = ''
                for f_n in [f for f in listdir(downl_path) if isfile(join(downl_path, f))]:
                    if 'pdf' in f_n:
                        pdf_saved_path = f"{downl_path}/{f_n}"
                        break
                if pdf_saved_path:
                    reader = PdfReader(pdf_saved_path)
                    text = ""
                    for page in reader.pages:
                        text += page.extract_text() + "\n"
                    f.write(text)
                    os.remove(pdf_saved_path)
        
        else:
            response = requests.get(url)
            if '200' in str(response):
                text_html = response.text
                clean_text = html2text.html2text(text_html)
                f.write(clean_text)
            else:
                driver = webdriver.Chrome(r'C:\Users\Anton\Desktop\chromedriver.exe')
                driver.get(url)
                text_html = driver.page_source
                clean_text = html2text.html2text(text_html)
                f.write(clean_text)
                driver.close()
            

C:\Users\Anton\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
<ipython-input-2-ecb60cca02bc>:38: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\Anton\Desktop\chromedriver.exe')
